<a href="https://colab.research.google.com/github/ClaraL27/NNDS-2023/blob/main/prove_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- hugging face: https://huggingface.co/datasets/go_emotions
- kaggle: https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp/data?select=test.txt
- kaggle code: https://www.kaggle.com/code/praveengovi/classify-emotions-in-text-with-bert


In [1]:
!pip -q install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00


In [ ]:
!pip -q install --upgrade keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━ 558.3/589.8 MB 951.4 kB/s eta 0:00:34

In [ ]:
import keras_nlp

In [ ]:
!pip -q install --upgrade keras

In [15]:
import keras_nlp
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import time

ModuleNotFoundError: No module named 'tensorflow.keras'

In [2]:
# To ensure reproducible results (as much as possible)
tf.keras.utils.set_random_seed(1234)

## Data loading

If kaggle is where the dataset is from

In [ ]:
# from google.colab import files
# files.upload()
# !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
# # Download the emotions dataset from kaggle
# !kaggle datasets download -d praveengovi/emotions-dataset-for-nlp

Dataset from hugging face

In [ ]:
from datasets import load_dataset

dataset = load_dataset("go_emotions", "raw", split=['train[:70%]', 'train[70%:85%]', 'train[85%:]'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/211225 [00:00<?, ? examples/s]

## Extract datasets

In [ ]:
train_data = dataset[0].to_tf_dataset()
val_data = dataset[1].to_tf_dataset()
test_data = dataset[2].to_tf_dataset()

In [ ]:
dict_emotions = {'admiration': 0,
                 'amusement': 1,
                 'anger': 2,
                 'annoyance': 3,
                 'approval': 4,
                 'caring': 5,
                 'confusion': 6,
                 'curiosity': 7,
                 'desire': 8,
                 'disappointment': 9,
                 'disapproval': 10,
                 'disgust': 11,
                 'embarrassment': 12,
                 'excitement': 13,
                 'fear': 14,
                 'gratitude': 15,
                 'grief': 16,
                 'joy': 17,
                 'love': 18,
                 'nervousness': 19,
                 'optimism': 20,
                 'pride': 21,
                 'realization': 22,
                 'relief': 23,
                 'remorse': 24,
                 'sadness': 25,
                 'surprise': 26,
                 'neutral': 27
                 }

In [ ]:
def preprocess(item):
  label = tf.math.argmax(list(item.values())[9:])
  return item['text'], label

In [ ]:
train_data_p = train_data.map(preprocess).shuffle(5000).batch(32)
val_data_p = val_data.map(preprocess).shuffle(5000).batch(32)
test_data_p = test_data.map(preprocess).shuffle(5000).batch(32)

In [ ]:
train_data_p.element_spec

(TensorSpec(shape=(None,), dtype=tf.string, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

**TODO**: remember to do some text pre-processing

## Train the base model


## Fine-Tuning with LoRA

Using RoBERTa as the model. \\
need to use preprocessor RoBERTa https://keras.io/api/keras_nlp/models/roberta/roberta_preprocessor/


In [ ]:
preprocessor = keras_nlp.models.RobertaPreprocessor.from_preset(
    "roberta_base_en"
)